In [1]:
# Install vector database
# ! pip install vectordb
# !pip install faiss-cpu
# !pip install faiss-gpu
# CONDA
# !conda install -c conda-forge faiss-gpu
# !conda install -c conda-forge transformers

## Import modules


In [1]:
import numpy as np
from utils import clean_dbs, get_all_feats, get_all_docs, reformat_keyframe, visualize
from vector_database import TextEmbedding, FrameDoc, VectorDB
from const import *
from bert_search import predict
# ~ 10 mins

Get all script...
Finished!
Initalize Tokenizer and BERT model


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Finished!
Set up Bert and faiss
Finished!


In [2]:
clean_dbs()
reformat_keyframe()

### Get all features files


In [3]:
all_feat_files = get_all_feats()
print(all_feat_files)
print(len(all_feat_files))

['../data/features/L01_V001.npy', '../data/features/L01_V002.npy', '../data/features/L01_V003.npy', '../data/features/L01_V004.npy', '../data/features/L01_V005.npy', '../data/features/L01_V006.npy', '../data/features/L01_V007.npy', '../data/features/L01_V008.npy', '../data/features/L01_V009.npy', '../data/features/L01_V010.npy', '../data/features/L01_V011.npy', '../data/features/L01_V012.npy', '../data/features/L01_V013.npy', '../data/features/L01_V014.npy', '../data/features/L01_V015.npy', '../data/features/L01_V016.npy', '../data/features/L01_V017.npy', '../data/features/L01_V018.npy', '../data/features/L01_V019.npy', '../data/features/L01_V020.npy', '../data/features/L01_V021.npy', '../data/features/L01_V022.npy', '../data/features/L01_V023.npy', '../data/features/L01_V024.npy', '../data/features/L01_V025.npy', '../data/features/L01_V026.npy', '../data/features/L01_V027.npy', '../data/features/L01_V028.npy', '../data/features/L01_V029.npy', '../data/features/L01_V030.npy', '../data/

### Create all the Docs


In [4]:
doc_list = get_all_docs(all_feat_files)

In [5]:
print(len(doc_list))
print(doc_list[100])

202148

            Video name: L01_V001
            Image path: ../data/keyframes/L01_V001\0101.jpg
            Keyframe Id: 101
            Actual keyframe idx: 11047
            Time: 441.88
            FPS: 25
            Metadata: {'publish_date': '01/12/2022', 'watch_url': 'https://youtube.com/watch?v=HNsRpkryGXA'}
          


## Filter by audio

In [6]:
SCRIPT_PATH = "../data/scripts/"
def check_script(file_content, keywords):
    for keyword in keywords:
        if (keyword not in file_content):
            return False
    return True
def filter_by_audio(result, keywords):
    for i in range(len(result) - 1, -1, -1):
        transcript_path = SCRIPT_PATH + result[i].video_name + ".txt"
        try:
            with open(transcript_path, 'r') as file:
                content = file.read()
                if (not check_script(content, keywords)):
                    result.pop(i)                 
        except FileNotFoundError:
            pass
    return result

## DEMO


### Create DB


In [7]:
DB = VectorDB()
DB.index(doc_list)

### Query


### Filter 1


In [11]:
results1 = DB.search(
    "Accident",
    1000,
)  # Nên lấy nhiều

In [12]:
visualize(filter_by_audio(results1[:100], []))

In [10]:
predict("gây tai nạn", results1, "predict.csv")